<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 4. Evaluate

In [1]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [2]:
data_dir = '/publication-tracking/benchmarking/CellTrackerGNN/data'
models = [
    'Fluo-N2DL-HeLa',
    'Fluo-N2DH-SIM+'
]

pattern = re.compile('\d{3}_GT')

node_match_threshold = 0.6

ctc_software = '/publication-tracking/benchmarking/CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [5]:
benchmarks = []

for m in models:
    print(f'------ {m} ------')
    res_subdir = os.path.join(data_dir, m)
    data_ids = [f.split('_')[0] for f in os.listdir(res_subdir) if pattern.fullmatch(f)]

    for data_id in data_ids:
        results = {
            'model': f'CellTrackerGNN {m}',
            'data_id': data_id
        }
        gt_dir = os.path.join(res_subdir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(res_subdir, f'{data_id}_RES')

        # Deepcell division benchmarking
        try:
            metrics = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir)
            results.update(metrics.stats)
        except ValueError:
            print('Issue with deepcell benchmarking of', data_id)
            
       # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), res_subdir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout
            
            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, m, data_id)
                print(outstring.decode('utf-8'))
        
        benchmarks.append(results)
        
df = pd.DataFrame(benchmarks)
df.to_csv('/publication-tracking/benchmarking/CellTrackerGNN/benchmarks.csv')

------ Fluo-N2DL-HeLa ------
missed node 1_29 division completely
missed node 3_23 division completely
29_29 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 47_17 division completely
missed node 6_36 division completely
missed node 6_17 division completely
missed node 8_18 division completely
10_19 out degree = 1, daughters mismatch.
15_17 out degree = 3, daughters mismatch.
17_6 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 19_20 division completely
missed node 29_17 division completely
missed node 60_34 division completely
missed node 76_7 division completely
77_16 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 83_18 division completely
1_48 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 6_16 division completely
10_13 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 23_66 division completely
24_11 out degree = 1, daughters mismatch.
missed node 32_35 divisio

In [6]:
df

,model,data_id,correct_division,mismatch_division,false_positive_division,false_negative_division,total_divisions,aa_tp,aa_total,te_tp,te_total,DET,SEG,TRA
0,CellTrackerGNN Fluo-N2DL-HeLa,003,4,1,12,3,8,1695,2055,1750,2122,0.846843,0.749611,0.846187
1,CellTrackerGNN Fluo-N2DL-HeLa,005,1,0,2,1,2,583,690,601,712,0.870787,0.769787,0.870175
2,CellTrackerGNN Fluo-N2DL-HeLa,012,5,4,347,7,16,7112,9796,7315,10097,0.830702,0.613603,0.826647
3,CellTrackerGNN Fluo-N2DL-HeLa,007,0,1,31,0,1,111,199,116,206,0.966019,0.880947,0.952784
4,CellTrackerGNN Fluo-N2DL-HeLa,008,5,6,100,9,20,4097,4945,4190,5056,0.875237,0.741574,0.874566
5,CellTrackerGNN Fluo-N2DL-HeLa,010,14,11,293,29,54,6719,9090,6924,9357,0.850540,0.609432,0.847588
6,CellTrackerGNN Fluo-N2DL-HeLa,011,2,4,76,9,15,2598,3833,2691,3975,0.730893,0.536113,0.729133
7,CellTrackerGNN Fluo-N2DL-HeLa,002,1,0,0,0,1,922,1075,946,1109,0.853021,0.737136,0.853646
8,CellTrackerGNN Fluo-N2DL-HeLa,004,1,0,13,0,1,324,521,335,540,0.624815,0.565398,0.626162
9,CellTrackerGNN Fluo-N2DL-HeLa,009,21,20,262,16,57,11498,14806,11786,15174,0.849011,0.674106,0.847008
